In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os

PROJECT_DPATH = os.path.abspath(os.pardir)
DATA_DPATH = os.path.join(PROJECT_DPATH, "data")

# for pip environment
sys.path.append(PROJECT_DPATH)

import torch
import torchvision

from ipywidgets import interact, IntSlider
import matplotlib.pyplot as plt
import numpy as np

from mnist_recognition.inference import Inference
from mnist_recognition.transforms import Invertor

## Загрузка обученной модели

In [ ]:
BATCH_SIZE = 64
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

checkpoint_dpath = os.path.join(PROJECT_DPATH, "checkpoints")
model_fpath = os.path.join(checkpoint_dpath, "best_valid_with_augmentations.pth")

infer = Inference.from_file(model_fpath, device= DEVICE)

## Загрузка тестовых данных

In [ ]:
test_data = torchvision.datasets.MNIST(
   DATA_DPATH, train=False, transform=Invertor(), download=True
)

# Создаём лоядеры данных.
# так как модель ожидает данные в определённой форме
test_dataloader=torch.utils.data.DataLoader(
    dataset=test_data, 
    batch_size=BATCH_SIZE,
    shuffle=False
)

print(f"Тестовая выборка содержит {len(test_data)} изображений")

In [ ]:
@interact
def show_inference(index = IntSlider(val=0, min=0, max=len(test_data)-1)):
    test_img, _true_label = test_data[index]
    test_img = np.array(test_img)
    pred_label = infer.get_prediction(test_img)

    plt.figure(figsize=[5, 5])
    plt.imshow(test_img, cmap="gray")
    plt.title(f"Предсказанный класс: {pred_label}")
    plt.show()